In [1]:
import os
import numpy as np
from sklearn import mixture
import pandas as pd
import re

In [2]:
#change path
path="..\\keystroke.csv"

In [3]:
import pandas
import numpy as np
data = pandas.read_csv(path)
subjects = data["subject"].unique()

In [4]:
subjects

array(['s002', 's003', 's004', 's005', 's007', 's008', 's010', 's011',
       's012', 's013', 's015', 's016', 's017', 's018', 's019', 's020',
       's021', 's022', 's024', 's025', 's026', 's027', 's028', 's029',
       's030', 's031', 's032', 's033', 's034', 's035', 's036', 's037',
       's038', 's039', 's040', 's041', 's042', 's043', 's044', 's046',
       's047', 's048', 's049', 's050', 's051', 's052', 's053', 's054',
       's055', 's056', 's057'], dtype=object)

In [5]:

mean_vector = []

In [6]:
from sklearn.mixture import GMM
import pandas 
#from EER_GMM import evaluateEER
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [7]:
def training(train):
    gmm = GMM(n_components = 2, covariance_type = 'diag', 
                    verbose = False )
    gmm.fit(train)
    return gmm

In [8]:
def testing(test_genuine, test_imposter, gmm):
    user_scores = []
    imposter_scores = []
    for i in range(test_genuine.shape[0]):
        j = test_genuine.iloc[i].values
        cur_score = gmm.score([j])
        user_scores.append(cur_score)

    for i in range(test_imposter.shape[0]):
        j = test_imposter.iloc[i].values
        cur_score = gmm.score([j])
        imposter_scores.append(cur_score)
        
    return user_scores, imposter_scores

In [9]:
def evaluateEERGMM(user_scores, imposter_scores):
    thresholds = range(20,51)
    array = np.zeros((len(thresholds),3))
    i = 0
    for th in thresholds:
        g_i = 0
        i_g = 0
        for score in user_scores:
            if score < th:
                g_i = g_i + 1
        for score in imposter_scores:    
            if score > th:
                i_g = i_g + 1

        FA = float(i_g) / len(imposter_scores) 
        FR = float(g_i) / len(user_scores)
        array[i, 0] = th
        array[i, 1] = FA
        array[i, 2] = FR
        i = i + 1
    
    for j in range(array.shape[0]):
        if array[j,1] < array[j,2]:
            thresh = (array[j,0] + array[j - 1, 0]) / 2
            break
    g_i = 0
    i_g = 0
    for score in user_scores:
        if score < thresh:
            g_i = g_i + 1
    for score in imposter_scores:    
        if score > thresh:
            i_g = i_g + 1

    FA = float(i_g) / len(imposter_scores) 
    FR = float(g_i) / len(user_scores)
    return (FA + FR) /2
    

In [10]:
def evaluate(subjects):    
    eers = []

    for subject in subjects:        
        genuine_user_data = data.loc[data.subject == subject, \
                                     "H.period":"H.Return"]
        imposter_data = data.loc[data.subject != subject, :]

        train = genuine_user_data[:200]
        test_genuine = genuine_user_data[200:]
        test_imposter = imposter_data.groupby("subject"). \
                             head(5).loc[:, "H.period":"H.Return"]

        gmm = training(train)
        user_scores, imposter_scores = testing(test_genuine, test_imposter, gmm)
        eers.append(evaluateEERGMM(user_scores, imposter_scores))
    return np.mean(eers)

In [11]:
print ("average EER for GMM detector:")

print(evaluate(subjects))

average EER for GMM detector:
0.13268627450980394


In [12]:
subject = subjects[0]
subject


's002'

In [13]:
genuine_user_data = data.loc[data.subject == subject, \
                             "H.period":"H.Return"]
genuine_user_data

,H.period,DD.period.t,UD.period.t,H.t,DD.t.i,UD.t.i,H.i,DD.i.e,UD.i.e,H.e,...,H.a,DD.a.n,UD.a.n,H.n,DD.n.l,UD.n.l,H.l,DD.l.Return,UD.l.Return,H.Return
0,0.1491,0.3979,0.2488,0.1069,0.1674,0.0605,0.1169,0.2212,0.1043,0.1417,...,0.1349,0.1484,0.0135,0.0932,0.3515,0.2583,0.1338,0.3509,0.2171,0.0742
1,0.1111,0.3451,0.2340,0.0694,0.1283,0.0589,0.0908,0.1357,0.0449,0.0829,...,0.1412,0.2558,0.1146,0.1146,0.2642,0.1496,0.0839,0.2756,0.1917,0.0747
2,0.1328,0.2072,0.0744,0.0731,0.1291,0.0560,0.0821,0.1542,0.0721,0.0808,...,0.1621,0.2332,0.0711,0.1172,0.2705,0.1533,0.1085,0.2847,0.1762,0.0945
3,0.1291,0.2515,0.1224,0.1059,0.2495,0.1436,0.1040,0.2038,0.0998,0.0900,...,0.1457,0.1629,0.0172,0.0866,0.2341,0.1475,0.0845,0.3232,0.2387,0.0813
4,0.1249,0.2317,0.1068,0.0895,0.1676,0.0781,0.0903,0.1589,0.0686,0.0805,...,0.1312,0.1582,0.0270,0.0884,0.2517,0.1633,0.0903,0.2517,0.1614,0.0818
5,0.1394,0.2343,0.0949,0.0813,0.1299,0.0486,0.0744,0.1412,0.0668,0.0863,...,0.1272,0.1534,0.0262,0.0858,0.2528,0.1670,0.0792,0.2971,0.2179,0.0591
6,0.1064,0.2069,0.1005,0.0866,0.1368,0.0502,0.0800,0.1407,0.0607,0.0789,...,0.1318,0.1204,-0.0114,0.0782,0.1999,0.1217,0.0879,0.2907,0.2028,0.0890
7,0.0929,0.1810,0.0881,0.0818,0.1378,0.0560,0.0747,0.1367,0.0620,0.0776,...,0.1322,0.1040,-0.0282,0.0821,0.2127,0.1306,0.1006,0.2776,0.1770,0.0679
8,0.0966,0.1797,0.0831,0.0771,0.1296,0.0525,0.0839,0.1425,0.0586,0.0755,...,0.1262,0.1403,0.0141,0.0787,0.2138,0.1351,0.0882,0.2868,0.1986,0.0634
9,0.1093,0.1807,0.0714,0.0731,0.1457,0.0726,0.0766,0.1241,0.0475,0.0813,...,0.1463,0.1162,-0.0301,0.1207,0.2281,0.1074,0.1204,0.3187,0.1983,0.0866


In [14]:
imposter_data = data.loc[data.subject != subject, :]
imposter_data

,subject,sessionIndex,rep,H.period,DD.period.t,UD.period.t,H.t,DD.t.i,UD.t.i,H.i,...,H.a,DD.a.n,UD.a.n,H.n,DD.n.l,UD.n.l,H.l,DD.l.Return,UD.l.Return,H.Return
400,s003,1,1,0.1333,0.2636,0.1303,0.1246,0.1737,0.0491,0.1330,...,0.1291,0.2234,0.0943,0.1090,0.3811,0.2721,0.1003,0.2997,0.1994,0.1035
401,s003,1,2,0.1307,0.3850,0.2543,0.1214,0.1908,0.0694,0.0956,...,0.1016,0.2365,0.1349,0.0781,0.2932,0.2151,0.1138,0.3201,0.2063,0.1035
402,s003,1,3,0.1143,0.2752,0.1609,0.1148,0.1439,0.0291,0.1468,...,0.1132,0.2104,0.0972,0.1351,0.2502,0.1151,0.1227,0.2883,0.1656,0.0800
403,s003,1,4,0.1328,0.2296,0.0968,0.0985,0.7023,0.6038,0.1222,...,0.1470,0.1824,0.0354,0.0892,0.3454,0.2562,0.1090,0.3079,0.1989,0.0958
404,s003,1,5,0.1217,0.1863,0.0646,0.1175,0.1584,0.0409,0.1175,...,0.1288,0.1421,0.0133,0.0618,0.2373,0.1755,0.0971,0.2632,0.1661,0.0987
405,s003,1,6,0.1320,0.1979,0.0659,0.1217,0.1523,0.0306,0.0964,...,0.1014,0.1542,0.0528,0.0871,0.1314,0.0443,0.1164,0.2783,0.1619,0.1008
406,s003,1,7,0.1235,0.2285,0.1050,0.0913,0.1594,0.0681,0.0895,...,0.1291,0.1183,-0.0108,0.0861,0.1331,0.0470,0.1043,0.2702,0.1659,0.1098
407,s003,1,8,0.1190,0.3878,0.2688,0.1032,0.1291,0.0259,0.1135,...,0.1006,0.1183,0.0177,0.0713,0.1581,0.0868,0.0929,0.2411,0.1482,0.0829
408,s003,1,9,0.0921,0.2741,0.1820,0.1172,0.5906,0.4734,0.1124,...,0.1109,0.1690,0.0581,0.0465,0.1114,0.0649,0.0887,0.3446,0.2559,0.0927
409,s003,1,10,0.1283,0.6059,0.4776,0.1246,0.1442,0.0196,0.1169,...,0.1048,0.1529,0.0481,0.1074,0.1823,0.0749,0.1029,0.2804,0.1775,0.0824


In [15]:
train = genuine_user_data[:200]


In [16]:
test_genuine = genuine_user_data[200:]
test_imposter = imposter_data.groupby("subject"). \
                     head(5).loc[:, "H.period":"H.Return"]

test_imposter

,H.period,DD.period.t,UD.period.t,H.t,DD.t.i,UD.t.i,H.i,DD.i.e,UD.i.e,H.e,...,H.a,DD.a.n,UD.a.n,H.n,DD.n.l,UD.n.l,H.l,DD.l.Return,UD.l.Return,H.Return
400,0.1333,0.2636,0.1303,0.1246,0.1737,0.0491,0.1330,0.1977,0.0647,0.0934,...,0.1291,0.2234,0.0943,0.1090,0.3811,0.2721,0.1003,0.2997,0.1994,0.1035
401,0.1307,0.3850,0.2543,0.1214,0.1908,0.0694,0.0956,0.2007,0.1051,0.1267,...,0.1016,0.2365,0.1349,0.0781,0.2932,0.2151,0.1138,0.3201,0.2063,0.1035
402,0.1143,0.2752,0.1609,0.1148,0.1439,0.0291,0.1468,0.4033,0.2565,0.1504,...,0.1132,0.2104,0.0972,0.1351,0.2502,0.1151,0.1227,0.2883,0.1656,0.0800
403,0.1328,0.2296,0.0968,0.0985,0.7023,0.6038,0.1222,0.6574,0.5352,0.1423,...,0.1470,0.1824,0.0354,0.0892,0.3454,0.2562,0.1090,0.3079,0.1989,0.0958
404,0.1217,0.1863,0.0646,0.1175,0.1584,0.0409,0.1175,0.5223,0.4048,0.0824,...,0.1288,0.1421,0.0133,0.0618,0.2373,0.1755,0.0971,0.2632,0.1661,0.0987
800,0.0937,0.4301,0.3364,0.0961,0.1803,0.0842,0.0808,0.1471,0.0663,0.0808,...,0.0855,0.1544,0.0689,0.0874,0.2233,0.1359,0.0990,0.3657,0.2667,0.0744
801,0.1183,0.6046,0.4863,0.0634,0.1371,0.0737,0.0927,0.1474,0.0547,0.0665,...,0.0871,0.1444,0.0573,0.1169,0.2465,0.1296,0.1006,0.4375,0.3369,0.0826
802,0.0998,0.2984,0.1986,0.0673,0.1257,0.0584,0.0934,0.1652,0.0718,0.0681,...,0.0876,0.1481,0.0605,0.0700,0.8577,0.7877,0.0895,0.3240,0.2345,0.0747
803,0.1027,0.4225,0.3198,0.0858,0.1175,0.0317,0.1048,0.1632,0.0584,0.0942,...,0.1138,0.1368,0.0230,0.0824,0.1639,0.0815,0.0932,0.3406,0.2474,0.0813
804,0.1072,0.2763,0.1691,0.0950,0.1850,0.0900,0.1003,0.1365,0.0362,0.0797,...,0.1143,0.1413,0.0270,0.0911,0.1792,0.0881,0.0990,0.2585,0.1595,0.0689


In [17]:
gmm = GMM(n_components = 2, covariance_type = 'diag', verbose = False )
gmm.fit(train)
gmm

GMM(covariance_type='diag', init_params='wmc', min_covar=0.001,
  n_components=2, n_init=1, n_iter=100, params='wmc', random_state=None,
  tol=0.001, verbose=False)

In [18]:
user_scores, imposter_scores = testing(test_genuine, test_imposter, gmm)
imposter_scores


[array([43.36860091]),
 array([41.93577126]),
 array([35.30394054]),
 array([-138.63295814]),
 array([-37.8400682]),
 array([45.49788504]),
 array([33.7441285]),
 array([18.04749482]),
 array([43.65917184]),
 array([47.78350343]),
 array([-456.88710513]),
 array([-45.24212821]),
 array([-125.87022192]),
 array([-21.17157901]),
 array([-8.31715561]),
 array([28.63314349]),
 array([23.86628144]),
 array([24.90927436]),
 array([20.07122887]),
 array([40.05906835]),
 array([41.7156826]),
 array([40.25138188]),
 array([42.52625161]),
 array([40.69067002]),
 array([40.79359065]),
 array([39.26507693]),
 array([12.44810289]),
 array([35.60079248]),
 array([51.11046171]),
 array([51.64812107]),
 array([43.94746012]),
 array([53.82182446]),
 array([50.46278185]),
 array([43.05054085]),
 array([40.51064183]),
 array([29.18505221]),
 array([11.58751868]),
 array([24.96542346]),
 array([-86.8840574]),
 array([21.39391494]),
 array([31.67465599]),
 array([54.04482691]),
 array([35.09101915]),
 arra

In [19]:
gmm.score([test_genuine.iloc[0].values])

array([56.22601788])

In [21]:
gmm.score([test_imposter.iloc[20].values])

array([41.7156826])